### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [2]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [3]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df[["rssi", "etx", "delay", "busy_fraction"]].values

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [5]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [6]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-59    1.1   17     0.20   1        Bom      1.00      
-60    1.4   6      0.10   1        Bom      1.00      
-78    6.8   19     0.49   0        Ruim     0.99      
-84    11.0  18     0.26   0        Ruim     1.00      
-64    3.2   7      0.20   1        Bom      0.99      
-43    1.5   4      0.32   1        Bom      1.00      
-58    1.3   4      0.22   1        Bom      1.00      
-57    1.2   14     0.17   1        Bom      1.00      
-62    1.0   7      0.11   1        Bom      1.00      
-69    1.8   5      0.22   1        Bom      0.99      
-62    3.1   7      0.34   1        Bom      0.97      
-57    1.0   14     0.30   1        Bom      1.00      
-49    2.6   18     0.19   1        Bom      1.00      
-78    4.4   42     0.82   0        Ruim     1.00      
-66    2.9   10     0.44   1        Bom      0.83      
-43    1.2   20     0.25   1        Bom      1.0

### Save

In [7]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
